In [1]:
from keras.layers import Input, Dropout, Dense, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
import pickle as pkl 
from sklearn.metrics import f1_score, classification_report
from layers.graph import SpectralGraphConvolution
from utils import *

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
def fix_labels(labels):
    for i in range(len(labels)):
        if labels[i][0] == "I":
            if i == 0 or labels[i-1][2:] != labels[i][2:]:
                labels[i] = "B-{}".format(labels[i][2:])
    return labels


def decode_labels(labels, idx2label):
    labels = np.array(labels)
    prediction_indices = labels.argmax(axis=1)
    prediction_labels = [idx2label[i] for i in prediction_indices]
    return prediction_labels


def predict_labels(predictions, actuals, idx2label):
    predictions_labels = []
    actuals_labels = []
    for i in range(len(predictions)):
#     for i in range(predictions.shape[0]):
        prediction = predictions[i]
        actual = actuals[i]
        prediction_labels = decode_labels(prediction, idx2label)
        prediction_labels = fix_labels(prediction_labels)
        actual_labels = decode_labels(actual, idx2label)
        predictions_labels.append(prediction_labels)
        actuals_labels.append(actual_labels)
    return predictions_labels, actuals_labels

In [3]:
def evaluate_metrics(y_true, y_pred):
        ## calc metric
    num_proposed = sum(1 for n in y_pred if n != 'O')
    num_correct = 0
    for i,j in zip(y_true,y_pred):
        if i != 'O' and i == j:
            num_correct +=1
    num_gold = sum(1 for n in y_true if n != 'O')
    print("num_proposed: ", num_proposed)
    print("num_correct: ", num_correct)
    print("num_gold: ", num_gold)
    try:
        precision = num_correct / num_proposed
    except ZeroDivisionError:
        precision = 1.0

    try:
        recall = num_correct / num_gold
    except ZeroDivisionError:
        recall = 1.0

    try:
        f1 = 2*precision*recall / (precision + recall)
    except ZeroDivisionError:
        if precision*recall==0:
            f1=1.0
        else:
            f1=0
    final = ".P%.2f_R%.2f_F%.2f" %(precision, recall, f1)
    print("precision=%.4f"%precision)
    print("recall=%.4f"%recall)
    print("f1=%.4f"%f1)
    print("final ",final)
    return f1

In [4]:
def f1_metric(y_true, y_pred):
        ## calc metric
    y_pred, y_true = predict_labels(
        y_pred, y_true, meta['idx2label'])
    for i in range(len(y_pred)):
        y_pred[i] = [x.split('-')[1] if '-' in x else x for x in y_pred[i]]
    for i in range(len(y_true)):
        y_true[i] = [x.split('-')[1] if '-' in x else x for x in y_true[i]]
    
    gt = []
    pr = []
    for i in range(len(y_pred)):
        gt.extend(y_pred[i])
    for i in range(len(y_true)):
        pr.extend(y_true[i])
        
    num_proposed = sum(1 for n in pr if n != 'O')
    num_correct = 0
    for i,j in zip(gt,pr):
        if i != 'O' and i == j:
            num_correct +=1
    num_gold = sum(1 for n in gt if n != 'O')
    try:
        precision = num_correct / num_proposed
    except ZeroDivisionError:
        precision = 1.0

    try:
        recall = num_correct / num_gold
    except ZeroDivisionError:
        recall = 1.0

    try:
        f1 = 2*precision*recall / (precision + recall)
    except ZeroDivisionError:
        if precision*recall==0:
            f1=1.0
        else:
            f1=0
    return f1

In [5]:
DATASET = 'conll2003'
EPOCHS = 4
LR = 5e-5
L2 = 0
DO = 0.5
BATCH_SIZE = 16

In [6]:
print("Loading dataset...")

A, X, Y, meta = pkl.load(open('pkl/' + DATASET + '.pkl', 'rb'))

print("Loading embedding matrix...")

embedding_matrix = pkl.load(
    open('pkl/' + DATASET + '.embedding_matrix.pkl', 'rb'))

print("Processing dataset...")

val_y = load_output(A, X, Y, 'val')
test_y = load_output(A, X, Y, 'test')

num_nodes = A['train'][0][0].shape[0]
num_relations = len(A['train'][0]) - 1
num_labels = len(meta['label2idx'])

print("Number of nodes: {}".format(num_nodes))
print("Number of relations: {}".format(num_relations))
print("Number of classes: {}".format(num_labels))

Loading dataset...
Loading embedding matrix...
Processing dataset...
Number of nodes: 124
Number of relations: 44
Number of classes: 8


In [7]:
# Define model inputs
X_in = Input(shape=(num_nodes, ))
A_in = [Input(shape=(num_nodes, num_nodes)) for _ in range(num_relations)]

In [9]:
print("Define model")
# Define model architecture
X_embedding = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], weights=[
                        embedding_matrix], trainable=False)(X_in)
H = SpectralGraphConvolution(256, activation='relu')([X_embedding] + A_in)
H = Dropout(DO)(H)
H = SpectralGraphConvolution(256, activation='relu')([H] + A_in)
H = Dropout(DO)(H)
output = Dense(num_labels, activation='softmax')(H)

# Compile model
model = Model(inputs=[X_in] + A_in, outputs=output)
model.compile(metrics=['acc'],loss='categorical_crossentropy', optimizer=Adam(lr=LR))
model.summary()

In [ ]:
# callbacks = [EarlyStopping(monitor='f1_metric', patience=2, verbose=0),
#              ModelCheckpoint(filepath='model.{loss:.2f}.h5', monitor='f1_metric', save_best_only=True, verbose=0)
#             ]

In [ ]:
EPOCHS = 50
for epoch in range(EPOCHS):

    print("=== EPOCH {} ===".format(epoch + 1))

    model.fit_generator(batch_generator(A, X, Y, 'train', batch_size=BATCH_SIZE),
                        steps_per_epoch=len(A['train'])//BATCH_SIZE, verbose=1)


    val_predictions = model.predict_generator(batch_generator(
        A, X, Y, 'val', batch_size=BATCH_SIZE), steps=len(A['val'])//BATCH_SIZE, verbose=1)
    val_predicted_labels, val_actual_labels = predict_labels(
        val_predictions, val_y, meta['idx2label'])

    for i in range(len(val_predicted_labels)):
        val_predicted_labels[i] = [x.split('-')[1] if '-' in x else x for x in val_predicted_labels[i]]
    for i in range(len(val_actual_labels)):
        val_actual_labels[i] = [x.split('-')[1] if '-' in x else x for x in val_actual_labels[i]]
    
    gt = []
    pr = []
    for i in range(len(val_predicted_labels)):
        gt.extend(val_predicted_labels[i])
    for i in range(len(val_actual_labels)):
        pr.extend(val_actual_labels[i])
        
    print("=== Validation Results ===")
    print("Weighted F1-score: ",f1_score(gt,pr, average = 'weighted'))
    print("Classification report:\n", classification_report(gt,pr))
    evaluate_metrics(gt, pr)

    test_predictions = model.predict_generator(batch_generator(
        A, X, Y, 'test', batch_size=BATCH_SIZE), steps=len(A['test']) // BATCH_SIZE, verbose=1)

    test_predicted_labels, test_actual_labels = predict_labels(
        test_predictions, test_y, meta['idx2label'])
    for i in range(len(test_predicted_labels)):
        test_predicted_labels[i] = [x.split('-')[1] if '-' in x else x for x in test_predicted_labels[i]]
    for i in range(len(test_actual_labels)):
        test_actual_labels[i] = [x.split('-')[1] if '-' in x else x for x in test_actual_labels[i]]

    print("=== Test Results ===")

    gt = []
    pr = []
    for i in range(len(test_predicted_labels)):
        gt.extend(test_predicted_labels[i])
    for i in range(len(test_actual_labels)):
        pr.extend(test_actual_labels[i])
    print("Weighted F1-score: ",f1_score(gt,pr, average = 'weighted'))
    print("Classification report:\n", classification_report(gt,pr))
    evaluate_metrics(gt, pr)